In [5]:
from chicktimer import ChickTimer

# 3 sec gap == 20BPM          ( marks start of CT sequence and after each 5 beep seperator )
# 3.8 sec gap == 15.789BPM    ( after each number in number pairs )
# 1.3 sec gap == 46.1538BPM   ( between each beep of 5 beep seperators )
# 0.8 sec gap == 75.00BPM     ( between each beep of data beeps )

class BeepStateMachine:

    gap_beep_rate_3sec: float = 20.00
    gap_beep_rate_3_8sec: float = 15.789
    gap_beep_rate_1_8sec: float = 46.13
    gap_beep_rate_0_8: float = 75.00

    number1_count: int
    number2_count: int
    seperator_count: int
    

    def __init__(self, inital_state:str = "BACKGROUND"):
        self.state = inital_state
        self.number1_count = 0
        self.number2_count = 0
        self.seperator_count = 0
        self.ct = ChickTimer()
        

    def process_input(self, BPM: float) -> None|ChickTimer:
        print(f"start of process - state is : {self.state}")
        if self.state == "BACKGROUND":
            if any(abs(BPM-background_beep_rate) < 0.5 for background_beep_rate in [80, 46, 30] ):
                # background beep rate, do nothing, return nothing, exit
                return
            if (abs(BPM - self.gap_beep_rate_3sec) < 0.5):
                # 3 secon pause encountered - indicates first set of digits
                self.state = "PAIR1_SET1"
                print(f"just got a 3sec gap. state is {self.state}")
                return
    
        if self.state == "PAIR1_SET1":
            # check expected BPM and if so count and increment
            if (abs(BPM - self.gap_beep_rate_0_8) < 0.5 ):
                self.number1_count += 1
                return
            # if BPM is 15.78 - exit as that was last beep of the set
            if (abs(BPM - self.gap_beep_rate_3_8sec ) < 0.5):
                # write the number of beeps to the data class
                #print(f"Pair 1 Set 1 : {self.beep_count}")
                #self.number1_count = 0
                self.state = "PAIR1_SET2"
                print(f"state is now {self.state}")
                return # this return needs to exit both loops?
       
        if self.state == "PAIR1_SET2":
            if (abs(BPM - self.gap_beep_rate_0_8) < 0.5 ):
                self.number2_count += 1
                return
            # if BPM is 15.78 - exit as last beep was last beep of that set
            if (abs(BPM - self.gap_beep_rate_3_8sec ) < 0.5):
                # write the number of beeps to the data class
                print(f"Pair 1 : {self.number1_count} & {self.number2_count}")
                self.ct.days_since_change_of_state = int(f"{self.number1_count}{self.number2_count}")
                print(self.ct)
                self.number1_count = 0
                self.number2_count = 0
                self.state = "SEPERATOR"
                return # this return needs to exit both loops?
            
        if self.state == "SEPERATOR":
            if (abs(BPM - self.gap_beep_rate_1_8sec) < 0.5):
                self.seperator_count += 1
                return
            if (beep_count == 5): # check for 5 beeps in seperator
                self.state = "PAIR2_SET1" # BUT IT COULD BE PAIR 2,3,4,5,6,7,8 - HOW DO I DETERMINE AT THIS POINT? check status of ct attributes?
                return
            
            
bsm = BeepStateMachine()
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.9 sec seperator





start of process - state is : BACKGROUND
just got a 3sec gap. state is PAIR1_SET1
start of process - state is : PAIR1_SET1
start of process - state is : PAIR1_SET1
start of process - state is : PAIR1_SET1
start of process - state is : PAIR1_SET1
state is now PAIR1_SET2
start of process - state is : PAIR1_SET2
start of process - state is : PAIR1_SET2
start of process - state is : PAIR1_SET2
start of process - state is : PAIR1_SET2
Pair 1 : 3 & 3
ChickTimer(days_since_change_of_state=33, days_since_hatch=0, days_since_desertion_alert=0, time_of_emergence=0, weeks_batt_life_left=0, activity_yesterday=0, activity_two_days_ago=0, mean_activity_last_four_days=0)
